# Part II - Model building

In this notebook, you will...
* Build a baseline model to compare all of your other models against
* Build an ensemble model (Gradient Boosting Regression)
* Build a simple neural network


You will also look at:
* Feature testing - how does your baseline model change with the various features

In [ ]:
import pandas as pd

from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

import itertools
import json
from typing import List, Dict

In [ ]:
current_dir = Path.cwd()
root_dir = current_dir.parent
data_dir = Path(current_dir, "data")

### Make the groups of sensors as needed

In [ ]:
ops = ['Unit_4_Power', 'Unit_4_Reactive Power', 'Turbine_Guide Vane Opening',
       'Turbine_Pressure Drafttube', 'Turbine_Pressure Spiral Casing',
       'Turbine_Rotational Speed', 'mode', 'Bolt_1_Steel tmp']

bolt_temp = ['Bolt_1_Steel tmp']

bolt_tensiles = ['Bolt_1_Tensile', 'Bolt_2_Tensile',  'Bolt_3_Tensile', 
                 'Bolt_4_Tensile', 'Bolt_5_Tensile',  'Bolt_6_Tensile']

bolt_torsions = ['Bolt_1_Torsion',  'Bolt_2_Torsion', 'Bolt_3_Torsion',
                 'Bolt_4_Torsion', 'Bolt_5_Torsion',  'Bolt_6_Torsion']

vibrations = ['lower_bearing_vib_vrt', 'turbine_bearing_vib_vrt'] 

### Read in the data. We will use the input dataset 2 since we know this is the most stable dataset.

In [ ]:
input_df = pd.read_parquet(Path(data_dir, "input_dataset-2.parquet"))

### Let's prepare the input dataframe for modelling. 
* If we have any NaNs in the model inputs and model outputs, we should either delete them or process them. Let's start by seeing how many NaNs there are.
* __```TODO``` - this work should be moved to the data exploration and preparation notebook__

In [ ]:
print("The original length of the dataframe is {} rows".format(input_df.shape))
print("If we drop NaNs in the subset of inputs and outputs, we end up with {} rows".format(
    input_df.dropna(subset=ops+bolt_tensiles).shape))

That still leaves us with an acceptable number of rows for training, so let's go ahead and drop the NaNs in the subset.

In [ ]:
input_df = input_df.dropna(subset=ops+bolt_tensiles)

We make the mode non-categorical for easier model training.

In [ ]:
input_df['mode'], codes = pd.factorize(input_df['mode'])

#### Prepare our data for training and testing.
First, we'll separate into X and y and then divide into train and test. We'll make the test size about 10%, using a random state of 0 and no shuffle so that we end up with data that is continuous in time.

Our ```X``` are the operating conditions.

In [ ]:
model_inputs = ops
X = input_df[model_inputs]

Our ```Y``` are the bolt tensiles.

In [ ]:
model_outputs = bolt_tensiles
y = input_df[model_outputs]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, shuffle=False)

#### Let's make a baseline model to evaluate against.

We'll use a Min Max Scaler and a Linear Regression model.

* Create the pipeline
* Fit the baseline model
* Predict using the trained baseline model
* Compare predictions and actuals using mean squared error and mean absolute error

In [ ]:
pipeline_baseline = make_pipeline(MinMaxScaler(), LinearRegression())

predictions = []
scores = {}

pipeline_baseline.fit(X_train, y_train)

y_pred = pipeline_baseline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("Mean Squared Error: {} \nMean Absolute Error: {} \nExplained Variance Score: {}".format(mse, mae, evs))

#### Is this good or bad? Let's visualize the actuals and predicted to see what it looks like.

In [ ]:
y_pred_df = pd.DataFrame(y_pred, index = y_test.index)
y_pred_df.columns=y_test.columns
    
fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(20, 20))

for t, ax in zip(bolt_tensiles, axes):
    ax.scatter(y_test.index, 
               y_test[t], label=t, color='r',
               s=1, alpha=0.1)

    ax.scatter(y_pred_df.index, 
               y_pred_df[t], label=t, color='b',
               s=1, alpha=0.1)

plt.tight_layout()
plt.show()

#### Let's make some helper functions to quickly train and score different models, and one to plot actuals vs predicted

In [ ]:
def model_train_score(X_train, X_test, y_train, y_test, model_pipeline):
    
    predictions = []
    for y_ in y_train:
        print("Training {}...".format(y_))
        model_pipeline.fit(X_train, y_train[y_])
        y_pred_ = model_pipeline.predict(X_test)
        predictions.append(pd.Series(y_pred_, index = X_test.index, name=y_))

    y_pred_df = pd.concat(predictions, axis=1)
    
    mse = mean_squared_error(y_test, y_pred_df)
    mae = mean_absolute_error(y_test, y_pred_df)
    evs = explained_variance_score(y_test, y_pred_df)
    
    #y_pred_df.columns=y_test.columns
    
    return [mse, mae, evs], y_pred_df

In [ ]:
def plot_actuals_vs_predicted(df_actuals, df_predictions, show=True, save_as=False):

    fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(20, 20))

    for t, ax in zip(bolt_tensiles, axes):
        ax.scatter(df_actuals.index, 
                   df_actuals[t], label=t, color='r',
                   s=1, alpha=0.1)

        ax.scatter(df_predictions.index, 
                   df_predictions[t], label=t, color='b',
                   s=1, alpha=0.1)

    plt.tight_layout()
    
    if show:
        plt.show()
    
    if save_as:
        plt.savefig(save_as)

### Feature engineering

There seems to be a bit of an offset between the predictions and the actuals. This might be accomodated by looking at a ```time_since_start``` feature. This should help to capture the linear increase we're observing over time.

In [ ]:
input_df["time_since_start"] = input_df.index - input_df.index[0]
input_df["time_since_start"] = input_df["time_since_start"].dt.total_seconds()

Does including this into the model train improve the performance?

In [ ]:
inputs = ops+["time_since_start"]
outputs = bolt_tensiles

X = input_df[inputs]
y = input_df[outputs]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, shuffle=False)

pipeline = make_pipeline(MinMaxScaler(), LinearRegression())

test = model_train_score(X_train, X_test, y_train, y_test, pipeline)
scores, predictions_df = model_train_score(X_train, X_test, y_train, y_test, pipeline)

print("Mean Squared Error: {} \nMean Absolute Error: {} \nExplained Variance Score: {}".format(scores[0], 
                                                                                               scores[1], 
                                                                                               scores[2]))

In [ ]:
plot_actuals_vs_predicted(y_test, predictions_df)

Much better - this seems to capture a lot of that offset we were seeing in the previous iteration. Let's also add a couple of other features and see if they allow us to minimize the gaps at the beginning of each start

### Additional Features that we can look at...

__```TODO``` - this should be moved to the data exploration and preparation__

* ```time_in_operation``` - this could help us to evaluate how time running affects the temperature and behaviour of the bolts
* ```time_since_last_stop``` - this could help us to evaluate how cold the system is when it's starting

In [ ]:
codes

The mode for operation is 0 and the mode for start is 1 (based on the codes generated during the factorization earlier).

We will then first identify each running window:


In [ ]:
# Find starts. Start = 1; So all starts will be where the difference == 1
input_df['temporary_mode_diff'] = input_df['mode'].diff()
input_df.loc[input_df.temporary_mode_diff == 1, "temporary_operation_starts_ends"] = "start"

# Find ends. Shift the temporary_mode_diff up one, because right before each start is the end of the last operation
input_df['temporary_mode_diff_ends'] = input_df['temporary_mode_diff'].shift(-1)
input_df.loc[input_df.temporary_mode_diff_ends == 1, "temporary_operation_starts_ends"] = "end"

# Wherever there is an end of operation, save the time. Wherever there is a start, save the time.
input_df.loc[input_df.temporary_operation_starts_ends == "end", "end_of_operation"] = input_df.index.to_series()
input_df.loc[input_df.temporary_operation_starts_ends == "start", "start_of_operation"] = input_df.index.to_series()

# We forward fill the end of operation so we can say how long it's been since the last operational mode.
input_df["time_since_last_stop"] = input_df["end_of_operation"].ffill()

# We forward fill the start of operation so we can say how long it's been since the current operational mode started.
input_df["time_in_operation"] = input_df["start_of_operation"].ffill()

# Subtract this time from the index to get the actual time since last operation
input_df["time_since_last_stop"] = input_df.index.to_series() - input_df["time_since_last_stop"]

# Subtract the time from the index to get the actual time running
input_df["time_in_operation"] = input_df.index.to_series() - input_df["time_in_operation"]

# Convert to seconds
input_df["time_since_last_stop"] = input_df["time_since_last_stop"].dt.total_seconds()
input_df["time_in_operation"] = input_df["time_in_operation"].dt.total_seconds()

## Multiple models & scaling methods

### REGRESSION

#### Let's look at 3 scaling methods (MinMax, Standard, and None) in conjunction with 2 prediction methods (a linear and an ensemble approach)

In [ ]:
scaling_methods = [MinMaxScaler(),
                   StandardScaler(),
                   None]

prediction_methods = [LinearRegression(),
                      GradientBoostingRegressor()]

Create tuples of all possible combinations of the scaling and prediction methods.

In [ ]:
pipeline_combinations = list(itertools.product(scaling_methods, prediction_methods))

# Let's just print the list to verify what comes out.
pipeline_combinations

Create sklearn pipelines for the combinations. 

__Note:__ the ```GradientBoostingRegressor``` takes some time to run, so you can first evaluate the effect of the scalers against the baseline model ```LinearRegression```, before you start looking at different prediction methods. 

In [ ]:
all_pipelines = [make_pipeline(*x) for x in pipeline_combinations]

In [ ]:
model_output_dir = Path(current_dir, "model_outputs")

In [ ]:
all_scores = {}
all_predictions = []
for p in all_pipelines:
    p_name = "_".join(p.named_steps.keys())
    print(p_name)
    
    scores, predictions_df = model_train_score(X_train, X_test, y_train, y_test, p)
    all_scores[p_name] = scores
    print(all_scores)
    
    # Save the predictions and the plots
    predictions_df.to_csv(Path(model_output_dir, "{}.csv".format(p_name)))
    
    plot_actuals_vs_predicted(y_test, 
                              predictions_df, 
                              show=False, 
                              save_as = Path(model_output_dir, "{}.jpeg".format(p_name)))

In [ ]:
# Print the scores
pd.DataFrame(all_scores).T.sort_values(by=1)

### LSTM

#### ...